## StyloMetrix Quick Tutorial 

### Do imports

In [1]:
import spacy
import stylo_metrix
from stylo_metrix.structures import CustomMetric, MetricsGroup
from stylo_metrix.utils import ratio

import warnings
warnings.filterwarnings('ignore')

### Initialize packages
Load spaCy model and add StyloMetrix to pipe with the name `stylo_metrix`:

In [2]:
nlp = spacy.load("pl_nask")
nlp.add_pipe("stylo_metrix")

ImportError: tokenizers>=0.10.1,<0.11 is required for a normal functioning of this module, but found tokenizers==0.12.1.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git master

### Use nlp on text
It will **automatically calculate the vector** and store it as `doc._.stylo_metrix_vector` attribute.

In [ ]:
txt = "Utwierdziła się w przekonaniu, że słusznie zrobiła, podejmując decyzję o odpuszczeniu\
       turnieju w Madrycie. Finału nie traktowała, jakby to był jakiś specjalny mecz. – Bardzo\
       się cieszę z tego jak podchodzę do finałów, że traktuję te mecze, jak każde inne. Nie\
       dałabym rady bez mojego sztabu, bardzo pomaga mi Daria w utrzymaniu nastawienia. Cieszę\
       się więc, że są ze mną i że każdy mecz gramy wspólnie na sto procent – zakończyła Świątek,\
       która już myśli o stracie na turnieju Roland Garros."
doc = nlp(txt)
doc._.stylo_metrix_vector

An alias for `doc._.stylo_metrix_vector` is `doc._.smv` for convenience. Let's print first 3 values:

In [ ]:
for metric in doc._.smv[:3]:
    print(metric)

### Create custom metrics
All you need is a **function that returns metric's value** decorated with `@CustomMetric()`:

In [ ]:
@CustomMetric("Liczba niepustych tokenów")
def METRIC(doc):
    result = doc._.n_tokens
    return result

You can use all **spaCy features** as well as our **built-in extensions and functions** inside code.

`token._.pos` is a built-in extension that stores data of general parts of speech. It equals `v` for verbs.

`doc._.n_tokens` is a built-in extension that stores number of non-empty tokens. 

`ratio(value1, value2)` is a built-in function to calculate ratio of two numbers or return 0 if `value2` is 0.

You can optionally add useful **debug information** in return statement. It will be stored along with vector data.

In [ ]:
@CustomMetric("Występowanie czasowników w 3 os. l. poj.")
def VERBS_3S(doc):
    verbs = [token for token in doc
            if token._.pos == "v" and token._.verb_person == "s3"]
    result = ratio(len(verbs), doc._.n_tokens)
    debug = {"verbs": verbs, "n_tokens": doc._.n_tokens}
    return result, debug

VERBS_3S(nlp("Utwierdziła się w przekonaniu, że słusznie zrobiła, podejmując decyzję o odpuszczeniu turnieju w Madrycie."))

Let's try a quick **test** on this metric. Each line contains a sentence and an expected (manually calculated) value:

In [ ]:
tests = (
    ("3/10", "Dział i wydział znał, słyszał i cieszy się."),
    ("4/9", "Klaszcze, piszczy, śpiewa i jeździ TIR-em."),
    ("3/6", "Przybyła, zobaczyła, zwycięży."),
    ("2/9", "Będzie się starał napisać dużo i niemało przeczytać."),
    ("2/7", "Zrobił śniadanie i posprząta stary radykał."),
)

print(f"expected received difference sentence")
for exp, sent in tests:
    expected = eval(exp)
    received = VERBS_3S(nlp(sent))["value"]
    difference = abs(expected - received)
    print(f"{expected:<8.2} {received:<8.2} {difference:<10.2} {sent}")

### Create groups
Put custom metrics in groups to manage them. You can perform **list-like operations** on groups.

In [ ]:
group = MetricsGroup(METRIC, VERBS_3S)
group

You can also import **built-in groups of metrics**.

In [ ]:
from stylo_metrix.metrics.pl import verbs_tenses_group, verbs_aspects_group
large_group = group + verbs_tenses_group + verbs_aspects_group
large_group

Assign new group as `nlp.metrics_group` attribute and recalculate the doc information to obtain new vector with **updated values**.

In [ ]:
nlp.metrics_group = large_group
doc = nlp(txt)
doc._.stylo_metrix_vector

Let's print first three metrics for the new group:

In [ ]:
for v in doc._.smv[:3]:
    print(v)

*source of input texts: https://www.onet.pl/sport/onetsport/iga-swiatek-ons-jabeur-iga-swiatek-zalala-sie-lzami-opowiedziala-o-tym/xkn97dq,d87b6cc4*